<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Title" data-toc-modified-id="Title-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Title</a></span><ul class="toc-item"><li><span><a href="#Business-Understanding-and-Business-Problem" data-toc-modified-id="Business-Understanding-and-Business-Problem-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Business Understanding and Business Problem</a></span><ul class="toc-item"><li><span><a href="#Cost-of-Errors" data-toc-modified-id="Cost-of-Errors-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Cost of Errors</a></span></li></ul></li><li><span><a href="#Data-Understanding" data-toc-modified-id="Data-Understanding-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Data Understanding</a></span><ul class="toc-item"><li><span><a href="#Initial-Data" data-toc-modified-id="Initial-Data-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Initial Data</a></span></li></ul></li><li><span><a href="#Exploratory-Data-Analysis" data-toc-modified-id="Exploratory-Data-Analysis-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Exploratory Data Analysis</a></span><ul class="toc-item"><li><span><a href="#Exploring-links_df" data-toc-modified-id="Exploring-links_df-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Exploring <code>links_df</code></a></span></li><li><span><a href="#Exploring-movies_df" data-toc-modified-id="Exploring-movies_df-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Exploring <code>movies_df</code></a></span></li><li><span><a href="#Exploring-ratings_df" data-toc-modified-id="Exploring-ratings_df-1.3.3"><span class="toc-item-num">1.3.3&nbsp;&nbsp;</span>Exploring <code>ratings_df</code></a></span></li><li><span><a href="#Exploring-tags_df" data-toc-modified-id="Exploring-tags_df-1.3.4"><span class="toc-item-num">1.3.4&nbsp;&nbsp;</span>Exploring <code>tags_df</code></a></span></li><li><span><a href="#Creating-a-New-DataFrame-with-Relevant-Data" data-toc-modified-id="Creating-a-New-DataFrame-with-Relevant-Data-1.3.5"><span class="toc-item-num">1.3.5&nbsp;&nbsp;</span>Creating a New DataFrame with Relevant Data</a></span><ul class="toc-item"><li><span><a href="#Additional-EDA-that-likely-can-be-dropped-for-final-notebook" data-toc-modified-id="Additional-EDA-that-likely-can-be-dropped-for-final-notebook-1.3.5.1"><span class="toc-item-num">1.3.5.1&nbsp;&nbsp;</span><strong>Additional EDA that likely can be dropped for final notebook</strong></a></span></li></ul></li></ul></li><li><span><a href="#Modeling" data-toc-modified-id="Modeling-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Modeling</a></span></li><li><span><a href="#First-Baseline-Model" data-toc-modified-id="First-Baseline-Model-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>First Baseline Model</a></span></li><li><span><a href="#Model-KNN-with-Z-Score" data-toc-modified-id="Model-KNN-with-Z-Score-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Model KNN with Z-Score</a></span></li><li><span><a href="#Model-with-SVD" data-toc-modified-id="Model-with-SVD-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Model with SVD</a></span></li></ul></li><li><span><a href="#Creating-a-New-User" data-toc-modified-id="Creating-a-New-User-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Creating a New User</a></span><ul class="toc-item"><li><span><a href="#Final-Model" data-toc-modified-id="Final-Model-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Final Model</a></span></li><li><span><a href="#Results/Conclusions" data-toc-modified-id="Results/Conclusions-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Results/Conclusions</a></span><ul class="toc-item"><li><span><a href="#Further-Exploration/Questions" data-toc-modified-id="Further-Exploration/Questions-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Further Exploration/Questions</a></span></li></ul></li></ul></li></ul></div>

insert header

# Title

## Business Understanding and Business Problem

### Cost of Errors

## Data Understanding

In [43]:
#imports grabbed from lecture
from random import gauss as gs, uniform as uni, seed
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression

from surprise import Reader, Dataset
from surprise.model_selection import train_test_split
from surprise.prediction_algorithms import knns, SVD, BaselineOnly
from surprise.similarities import cosine, msd, pearson
from surprise import accuracy

### Initial Data

In [2]:
#the four csv files provided
links_df = pd.read_csv('data/links.csv')
movies_df = pd.read_csv('data/movies.csv')
ratings_df = pd.read_csv('data/ratings.csv')
tags_df = pd.read_csv('data/tags.csv')

In [3]:
links_df.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [4]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [6]:
tags_df.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [7]:
sim_cos = {'name':'msd', 'user_based':False}

## Exploratory Data Analysis

### Exploring `links_df`

In [8]:
links_df.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


This file is a key to merge movie identifiers with IMDB Database and The Movie DataBase. We will be focusing on the MovieLens database for this analysis and will not need this file.

### Exploring `movies_df`

In [9]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [10]:
movies_df.shape

(9742, 3)

In [11]:
movies_df.isna().sum()

movieId    0
title      0
genres     0
dtype: int64

In [12]:
movies_df['title'].nunique()

9737

In [13]:
movies_df['title'].value_counts().head(10)

War of the Worlds (2005)                  2
Confessions of a Dangerous Mind (2002)    2
Emma (1996)                               2
Eros (2004)                               2
Saturn 3 (1980)                           2
With Six You Get Eggroll (1968)           1
Heavy Metal (1981)                        1
The Man from Nowhere (2010)               1
Ernest Scared Stupid (1991)               1
Ratchet & Clank (2016)                    1
Name: title, dtype: int64

There are 9737 unique movie titles. Repeats for 5 movies:
- Emma (1996)                                   2
- Saturn 3 (1980)                               2
- Eros (2004)                                   2
- Confessions of a Dangerous Mind (2002)        2
- War of the Worlds (2005)                      2

This file will be very helpful for our analysis. It provides us with the title of movie in relation to its unique identifier and lets us know the genre categories that the movie would fall under.

### Exploring `ratings_df`

In [14]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [15]:
ratings_df.isna().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [16]:
# The 10 movies with the most reviews
ratings_df.groupby('movieId').count().sort_values('userId', ascending=False).head(10)

,userId,rating,timestamp
movieId,,,
356,329,329,329
318,317,317,317
296,307,307,307
593,279,279,279
2571,278,278,278
260,251,251,251
480,238,238,238
110,237,237,237
589,224,224,224


In [17]:
# the 10 movies with the highest ratings
ratings_df.groupby('movieId').mean().sort_values('rating', ascending=False).head(10)

,userId,rating,timestamp
movieId,,,
88448,483.0,5.0,1.315438e+09
100556,462.0,5.0,1.456151e+09
143031,89.0,5.0,1.520409e+09
143511,105.0,5.0,1.526207e+09
143559,89.0,5.0,1.520410e+09
6201,474.0,5.0,1.100120e+09
102217,63.0,5.0,1.443200e+09
102084,380.0,5.0,1.493422e+09
6192,182.0,5.0,1.063275e+09


This will be the primary datafile for this analysis. It includes relevant information including `userId`, `rating`, and `movieId`. `timestamp` is not relevant for this analysis and will be dropped later on.

### Exploring `tags_df`

In [18]:
tags_df.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [19]:
tags_df.isna().sum()

userId       0
movieId      0
tag          0
timestamp    0
dtype: int64

In [20]:
# users with the most comments on movies
tags_df.groupby('userId').count().sort_values('tag', ascending=False).head(10)

,movieId,tag,timestamp
userId,,,
474,1507,1507,1507
567,432,432,432
62,370,370,370
599,323,323,323
477,280,280,280
424,273,273,273
537,100,100,100
125,48,48,48
357,45,45,45


This will be helpful for analysis as it provides keyword insights to each of the films. `timestamp` will not be helpful for this analysis and will be dropped later.

### Creating a New DataFrame with Relevant Data

In [21]:
# Dropping timestamp from ratings
new_ratings_df = ratings_df.drop('timestamp', axis=1).copy()

In [22]:
# Merging the ratings data with movie titles and genres
merged_df = new_ratings_df.merge(movies_df, how='outer', on='movieId')

In [23]:
merged_df.head()

,userId,movieId,rating,title,genres
0,1.0,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5.0,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7.0,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15.0,1,2.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17.0,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [24]:
# Checking for nulls after merging
merged_df.isna().sum()

userId     18
movieId     0
rating     18
title       0
genres      0
dtype: int64

In [25]:
# 18 movies do not have any reviews 
merged_df.tail(20)

,userId,movieId,rating,title,genres
100834,610.0,163937,3.5,Blair Witch (2016),Horror|Thriller
100835,610.0,163981,3.5,31 (2016),Horror
100836,NaN,1076,NaN,"Innocents, The (1961)",Drama|Horror|Thriller
100837,NaN,2939,NaN,Niagara (1953),Drama|Thriller
100838,NaN,3338,NaN,For All Mankind (1989),Documentary
100839,NaN,3456,NaN,"Color of Paradise, The (Rang-e khoda) (1999)",Drama
100840,NaN,4194,NaN,I Know Where I'm Going! (1945),Drama|Romance|War
100841,NaN,5721,NaN,"Chosen, The (1981)",Drama
100842,NaN,6668,NaN,"Road Home, The (Wo de fu qin mu qin) (1999)",Drama|Romance
100843,NaN,6849,NaN,Scrooge (1970),Drama|Fantasy|Musical


In [26]:
# Dropping nulls in data
merged_df.dropna(inplace=True)

In [27]:
merged_df.isna().sum()

userId     0
movieId    0
rating     0
title      0
genres     0
dtype: int64

In [28]:
merged_df.shape

(100836, 5)

In [29]:
merged_df['movieId'].nunique()

9724

We have a cleaned dataframe with 100836 reviews for 9724 movies.

#### **Additional EDA that likely can be dropped for final notebook**

In [30]:
# creating a sparse matrix pivot table that has all users as the index and all movies as the columns
pivot_test = merged_df.pivot_table(index='userId', columns='title', values='rating')

In [31]:
pivot_test.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Modeling

In [32]:
# Adjusting the dataframe for modeling purposes
model_df = merged_df.drop(['title', 'genres'], axis=1)

In [33]:
# Contains all movie ratings and the unique identifiers for user and movie
model_df.head()

,userId,movieId,rating
0,1.0,1,4.0
1,5.0,1,4.0
2,7.0,1,4.5
3,15.0,1,2.5
4,17.0,1,4.5


In [34]:
# Converting model_df to surprise accessible data
reader = Reader()
data = Dataset.load_from_df(model_df,reader)

In [35]:
trainset, testset = train_test_split(data, test_size=.2, random_state=42)

In [36]:
print('Number of users: ', trainset.n_users, '\n')
print('Number of items: ', trainset.n_items)

Number of users:  610 

Number of items:  8982


## First Baseline Model

Setting up a baseline model

In [56]:
baseline = BaselineOnly()
baseline.fit(trainset)

Estimating biases using als...


In [57]:
base_predictions = baseline.test(testset)

In [58]:
accuracy.rmse(base_predictions)

RMSE: 0.8734


0.8734341181528673

## Model KNN with Z-Score

In [59]:
knn_z = knns.KNNWithZScore(random_state=42)
knn_z.fit(trainset=trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [60]:
knn_z_predictions = knn_z.test(testset)

In [61]:
accuracy.rmse(knn_z_predictions)

RMSE: 0.8963


0.8962709994505563

## Model with SVD

In [62]:
svd = SVD(random_state=42)
svd.fit(trainset)

In [63]:
svd_predictions = svd.test(testset)

In [64]:
accuracy.rmse(svd_predictions)

RMSE: 0.8757


0.8757353176838333

# Creating a New User

In [71]:
merged_df.head()

,userId,movieId,rating,title,genres
0,1.0,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5.0,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7.0,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15.0,1,2.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17.0,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [72]:
merged_df['userId'].max()

610.0

In [73]:
new_user_df = merged_df.copy()

In [77]:
new_user_df['username'] = np.nan

In [78]:
new_user_df

,userId,movieId,rating,title,genres,username
0,1.0,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,NaN
1,5.0,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,NaN
2,7.0,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,NaN
3,15.0,1,2.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,NaN
4,17.0,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,NaN
...,...,...,...,...,...,...
100831,610.0,160341,2.5,Bloodmoon (1997),Action|Thriller,NaN
100832,610.0,160527,4.5,Sympathy for the Underdog (1971),Action|Crime|Drama,NaN
100833,610.0,160836,3.0,Hazard (2005),Action|Drama|Thriller,NaN
100834,610.0,163937,3.5,Blair Witch (2016),Horror|Thriller,NaN


In [94]:
new_user_df[(new_user_df['userId'] == 1) & (new_user_df['movieId'] == 1)]['userId']

0    1.0
Name: userId, dtype: float64

In [97]:
def new_user():
    answer = input('Are you a new user? (yes/no): ')
    if answer == 'yes':
        ok_to_move_on = False
        while ok_to_move_on is False:
            new_user_id = (new_user_df['userId'].max() + 1)
            username = input('Please enter a unique username: ')
            if username not in new_user_df['username']:
                password = input('Please enter a password: ')
                ok_to_move_on = True
            else: 
                print('Sorry, that name is taken.')
    if answer == 'no':
        ok_to_move_on = False
        while ok_to_move_on is False:
            username = input('Please enter your username: ')
            if username in new_user_df['username']:
                password = input('Please enter your password: ')
                if new_user_df[(new_user_df['userId'] == 1) & (new_user_df['movieId'] == 1)]:
                    user_id = new_user_df[(new_user_df['userId'] == 1) & (new_user_df['movieId'] == 1)]['userId']
                    ok_to_move_on = True
                else:
                    print('Sorry, that username and password is not in the system.')
            else:
                print('Sorry, that username is not in the system.')

In [98]:
new_user()

Are you a new user? (yes/no): no
Please enter your username: bill
Sorry, that username is not in the system.
Please enter your username: ted
Sorry, that username is not in the system.


KeyboardInterrupt: Interrupted by user

## Final Model

## Results/Conclusions

### Further Exploration/Questions